# Aplicaciones a Problemas Directos

**Por:** David Ortiz, Rodrigo Salas  
**Edición:** David Ortiz, Tabita Catalán, Tomás Banduc

### Introducción
En esta lección extendemos el uso de PINNs desde EDOs hacia **ecuaciones diferenciales parciales (EDPs)**. Nos enfocaremos en un modelo de difusión lineal: la **ecuación de calor 1D**, con el objetivo de estudiar cómo una PINN puede aproximar soluciones espacio–temporales y cómo se construye la función de pérdida cuando la física está dada por una EDP.

### Resumen de la actividad
Se implementa una PINN para resolver la ecuación de calor en una dimensión espacial. El objetivo es consolidar el flujo de trabajo para EDPs lineales en un esquema **data-free** (sin datos observados), utilizando una solución analítica como referencia.

### Objetivos de la actividad
Al finalizar esta actividad, serás capaz de:
- Formular un problema de EDP y construir una solución analítica de referencia.  
- Entrenar una PINN para resolver una EDP lineal con un enfoque *data-free*.  

### Descripción matemática del problema
Consideramos el modelo de difusión unidimensional (ecuación de calor):

$$
\frac{\partial u}{\partial t} = \kappa \frac{\partial^2 u}{\partial x^2} + f(t,x),
\qquad x \in [-1,1], \quad t \in [0,2], \quad \kappa \in \mathbb{R}.
$$

Aquí $u(t,x)$ representa una cantidad física (temperatura, concentración, voltaje, etc.), $\kappa$ es el coeficiente de difusión y $f(t,x)$ es un término fuente.

Para construir un caso con solución cerrada, fijamos $\kappa = 1$ y proponemos

$$
u(t,x) = e^{-t}\sin(\pi x).
$$

Al sustituir en la EDP se obtiene el siguiente problema (con condiciones iniciales y de borde) cuya solución exacta es la función propuesta:

$$
\begin{aligned}
\text{EDP:}\quad & \frac{\partial u}{\partial t}
= \frac{\partial^2 u}{\partial x^2}
- e^{-t}\big(\sin(\pi x) - \pi^2\sin(\pi x)\big),
&& x \in [-1, 1],\; t \in [0, 2], \\
\text{CI:}\quad & u(0,x) = \sin(\pi x),
&& x \in [-1, 1], \\
\text{CB:}\quad & u(t,-1) = u(t,1) = 0,
&& t \in [0, 2], \\
\text{Solución:}\quad & u(t,x) = e^{-t}\sin(\pi x). &&
\end{aligned}
$$

## Flujo de trabajo
1. Visualizar la solución analítica en una malla uniforme.  
2. Muestrear el dominio espacio–temporal para entrenar la PINN.  
3. Entrenar la PINN siguiendo las 6 etapas vistas, definiendo explícitamente la función de pérdida.  

## Recordemos el esquema básico
<img src="../figures/pinns_new_scheme.png" width="800" height="400">

- (1) formular el modelo matemático y su solución analítica;  
- (2) definir el dominio espacio–temporal;  
- (3) implementar una ANN como aproximador de $u(t,x)$;  
- (4) emplear diferenciación automática;  
- (5) diseñar la *loss* con términos físicos (residuo de la EDP y condición inicial/de borde) usando `autograd`;  
- (6) seleccionar un optimizador (Adam).  


### Configuración Inicial

Comenzamos importando módulos y definiendo algunas funciones de utilidad

In [ ]:
%matplotlib inline

In [ ]:
# NumPy para operaciones numéricas
import numpy as np
# PyTorch para construir y entrenar redes neuronales
import torch
import torch.nn as nn
import torch.optim as optim
# Matplotlib para graficar
import matplotlib.pyplot as plt
import matplotlib as mpl
# Time para medir tiempo de entrenamiento
import time
# Warnings para ignorar mensajes de advertencia
import warnings
warnings.filterwarnings("ignore")

from matplotlib import animation, rc
from scipy.stats import qmc

In [ ]:
# Setup (device + plots)
def get_device() -> str:
    return "cuda" if torch.cuda.is_available() else \
           "mps"  if torch.backends.mps.is_available() else "cpu"

def set_mpl_style(gray: str = "#5c5c5c") -> None:
    mpl.rcParams.update({
        "image.cmap": "viridis",
        "text.color": gray, "xtick.color": gray, "ytick.color": gray,
        "axes.labelcolor": gray, "axes.edgecolor": gray,
        "axes.spines.right": False, "axes.spines.top": False,
        "axes.formatter.use_mathtext": True, "axes.unicode_minus": False,
        "font.size": 15, "interactive": False, "font.family": "sans-serif",
        "legend.loc": "best", "text.usetex": False, "mathtext.fontset": "stix",
    })

device = get_device()
print(f"Using {device} device")
set_mpl_style()

# Definir pi en torch
torch.pi = torch.acos(torch.zeros(1)).item() * 2

def relative_l2_error(u_num: torch.Tensor, u_ref: torch.Tensor) -> torch.Tensor:
    return torch.norm(u_num - u_ref) / torch.norm(u_ref)

# Autodiff helper
def grad(outputs: torch.Tensor, inputs: torch.Tensor) -> torch.Tensor:
    """d(outputs)/d(inputs) with create_graph=True."""
    return torch.autograd.grad(
        outputs, inputs,
        grad_outputs=torch.ones_like(outputs),
        create_graph=True
    )[0]

def plot_comparison(u_true, u_pred, loss):
    u_hat = u_pred.detach().cpu().numpy()

    # --- Soluciones ---
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    for a, u, title in zip(
        ax,
        [u_true, u_hat],
        ['Analytic solution for diffusion', 'PINN solution for diffusion']
    ):
        im = a.imshow(u, extent=[-1, 1, 2, 0])
        a.set(title=title, xlabel=r'$x$', ylabel=r'$t$')
        fig.colorbar(im, ax=a, shrink=0.5)

    plt.tight_layout(); plt.show()

    # --- Error + entrenamiento ---
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    im = ax[0].imshow(np.abs(u_true - u_hat), extent=[-1, 1, 2, 0])
    ax[0].set(title=r'$|u(t,x) - u_{\mathrm{pred}}(t,x)|$',
              xlabel=r'$x$', ylabel=r'$t$')
    fig.colorbar(im, ax=ax[0], shrink=0.5)

    ax[1].plot(loss)
    ax[1].set(title='Training Progress',
              xlabel='Iteration', ylabel='Loss',
              xscale='log', yscale='log')
    ax[1].grid(True)

    plt.tight_layout(); plt.show()
    
def animate(x,t,U):

    # Primero, generar figura con subplot correspondiente
    fig, ax = plt.subplots(figsize = (10,6))
    plt.close()

    ax.set_title(r"heat solution $e^t sin(\pi x)$")

    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(np.floor(U.min()), np.ceil(U.max()))
    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"$u(x, t)$")

    # Inicializar etiqueta sin texto
    time_label = ax.text(1, 1, "", color = "black", fontsize = 12)
    # Inicializar gráfico sin datos
    line, = ax.plot([], [], color = "black", lw = 2)

    # Definir función de inicialización
    def init():
        line.set_data([], [])
        time_label.set_text("")
        return (line,)

    # Animar función. Esta función se llama secuencialmente con FuncAnimation
    def animate(i):
        line.set_data(x, U[i])
        time_label.set_text(f"t = {t[i]:.2f}")
        return (line,)

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                frames=len(t), interval=50, blit=True)

    return anim

## 1. Visualización de la solución Analítica
Utilizamos la solución $u(t,x) = e^{-t}\sin(\pi x)$ para el problema de difusión y la evaluamos en coordenadas específicas.

In [ ]:
# Número de muestras para espacio y tiempo.
# Obs: Podrían tomarse distintos valores de muestreo en tiempo y en espacio, pero se considera un solo valor por simplicidad
dom_samples = 100

# TODO: Defina función para solución analítica
# Indicación: utilice np.exp, np.sin y np.pi
def analytic_diffusion(x,t):
    u = ...
    return u

# Dominio espacial
x_lower = -1
x_upper = 1 
x = np.linspace(x_lower, x_upper, dom_samples)
# TODO: Definir límites del dominio temporal
t_lower = ... 
t_upper = ...
t = np.linspace(t_lower, t_upper, dom_samples)

# Mallado
X, T = np.meshgrid(x, t)
# Evaluar función en mallado
U = analytic_diffusion(X, T)

fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, T, U, cmap='viridis', edgecolor='k')

ax.set_xlabel('x')
ax.set_ylabel('t')
ax.set_zlabel('u(t, x)')
ax.set_title('3D Analytic Solution for Diffusion')

# Añadir la barra de color
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.show()

In [ ]:
# Correr animación y hacer display
anim = animate(x,t,U)
rc("animation", html="jshtml")
anim

## 2. Muestreo del dominio para el entrenamiento de la PINN

Para entrenar la PINN se muestrea el dominio espacio–temporal utilizando **Muestreo de Hipercubo Latino** (*Latin Hypercube Sampling*, LHS). Esta estrategia estratificada permite cubrir de manera uniforme el espacio de entrada, reduciendo la formación de aglomeraciones al dividir el dominio en subregiones y seleccionar un punto aleatorio en cada una de ellas.

En la implementación se utiliza `qmc.LatinHypercube` de `scipy.stats`, escalando posteriormente las muestras para que coincidan con los límites del dominio. Finalmente, los puntos muestreados se convierten a `torch.Tensor` para su uso directo en el entrenamiento de la PINN.


In [ ]:
# Muestreo con LHS
def collocation_lhs(l_bounds, u_bounds, n=100, plot=True):
    """
    n : int, opcional
        Número de puntos de colocación a generar. Por defecto es 100.
    l_bounds : tupla de float o int
        Límites inferiores del dominio (x_lower, t_lower).
    u_bounds : tupla de float o int
        Límites superiores del dominio (x_upper, t_upper).
    plot : bool, opcional (por defecto True)
        Si es True, muestra un scatter plot de los puntos generados. 
    """
    sampler = qmc.LatinHypercube(d=2)
    domain_xt = qmc.scale(sampler.random(n=n), l_bounds, u_bounds)

    # Interior (PDE)
    x_ten = torch.tensor(domain_xt[:, 0], requires_grad=True).float().view(-1, 1)
    t_ten = torch.tensor(domain_xt[:, 1], requires_grad=True).float().view(-1, 1)

    # Proyecciones desde los mismos puntos LHS
    x_ic, t_ic = domain_xt[:, 0], np.full(n, l_bounds[1])      # IC: t = t_min
    t_bc = domain_xt[:, 1]                                      # BC: usar los mismos t
    x_bcL, x_bcR = np.full(n, l_bounds[0]), np.full(n, u_bounds[0])  # x = x_min / x_max

    if plot:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.scatter(domain_xt[:, 0], domain_xt[:, 1], s=15, label='PDE (LHS)')
        ax.scatter(x_ic, t_ic, s=25, label=f'IC: t={l_bounds[1]} ')
        ax.scatter(x_bcL, t_bc, s=25, label=f'BC: x={l_bounds[0]} ')
        ax.scatter(x_bcR, t_bc, s=25, label=f'BC: x={u_bounds[0]} ')
        ax.set(title='Collocation points', xlabel=r'$x$', ylabel=r'$t$')
        ax.legend(loc='best'); ax.invert_yaxis()
        plt.tight_layout(); plt.show()

    return x_ten, t_ten

_ = collocation_lhs(l_bounds=(x_lower, t_lower), u_bounds=(x_upper, t_upper))

## Entrenamiento de la PINN

En esta etapa se entrena una PINN para aproximar la solución espacio–temporal $u(t,x)$ del problema de difusión, es decir,
$$
u_{\text{PINN}}(t,x;\Theta) \approx u(t,x),
$$
empleando la misma arquitectura y estrategia de entrenamiento utilizadas previamente.


### Función de pérdida informada por la física (usando autodiff)

Para entrenar la PINN se definen el residuo de la EDP, la condición inicial y las condiciones de borde. La solución exacta $u(t,x)$ se reemplaza por la salida de la red $u_{\text{PINN}}(t,x;\Theta)$:

$$
\begin{aligned}
f_{\text{pde}}(t,x;u_{\text{PINN}}) &=
\frac{\partial u_{\text{PINN}}}{\partial t}
- \frac{\partial^2 u_{\text{PINN}}}{\partial x^2}
+ e^{-t}\big(\sin(\pi x) - \pi^2 \sin(\pi x)\big), \\
g_{\text{ic}}(0,x;u_{\text{PINN}}) &=
u_{\text{PINN}}(0,x;\Theta) - \sin(\pi x), \\
h_{\text{bc1}}(t,-1;u_{\text{PINN}}) &=
u_{\text{PINN}}(t,-1;\Theta), \\
h_{\text{bc2}}(t,1;u_{\text{PINN}}) &=
u_{\text{PINN}}(t,1;\Theta).
\end{aligned}
$$

La función de pérdida informada por la física se construye utilizando el error cuadrático medio (MSE) como una combinación ponderada de los términos anteriores:

$$
\begin{aligned}
\mathcal{L}(\Theta) =\;&
\frac{\lambda_1}{N}\sum_i f_{\text{pde}}(t_i,x_i;u_{\text{PINN}})^2
+ \frac{\lambda_2}{N}\sum_i g_{\text{ic}}(0,x_i;u_{\text{PINN}})^2 \\
&+ \frac{\lambda_3}{N}\sum_i h_{\text{bc1}}(t_i,-1;u_{\text{PINN}})^2
+ \frac{\lambda_3}{N}\sum_i h_{\text{bc2}}(t_i,1;u_{\text{PINN}})^2,
\end{aligned}
$$

donde $\lambda_{1,2,3} \in \mathbb{R}^+$ son coeficientes de ponderación y $N$ es el número de muestras.

El entrenamiento de la PINN se formula como el siguiente problema de optimización:

$$
\min_{\Theta} \; \mathcal{L}(\Theta) \rightarrow 0,
$$

utilizando diferenciación automática (`torch.autograd`) para evaluar las derivadas necesarias en el residuo físico del modelo.

In [ ]:
# Definir clase de red neuronal con capas y neuronas especificadas por usuario
class NeuralNetwork(nn.Module):

    def __init__(self, hlayers):
        super(NeuralNetwork, self).__init__()

        layers = []
        for i in range(len(hlayers[:-2])):
            layers.append(nn.Linear(hlayers[i], hlayers[i+1]))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hlayers[-2], hlayers[-1]))

        self.layers = nn.Sequential(*layers)
        self.init_params()

    def init_params(self):
        """Inicialización de parámetros Xavier Glorot
        """
        def init_normal(m):
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight) # Xavier
        self.apply(init_normal)

    def forward(self, x):
        return self.layers(x)

A continuación se presenta el código completo:

In [ ]:
#===============================================================================
# ETAPA 1: DEFINICIÓN DE LOS PARÁMETROS (MODELO FÍSICO)
#===============================================================================
# TODO: definir límites del dominio
x_lower = ...
x_upper = ...
t_lower = ... 
t_upper = ...

l_bounds = [x_lower, t_lower]
u_bounds = [x_upper, t_upper]

Ncoll_points = 100

#===============================================================================
# ETAPA 2: DEFINICIÓN DEL DOMINIO 
#===============================================================================
x_ten, t_ten = collocation_lhs(l_bounds, u_bounds, n=Ncoll_points, plot=False)

#===============================================================================
# ETAPA 3: CREACIÓN DE LA RED NEURONAL SURROGANTE 
#===============================================================================
torch.manual_seed(123)

# hiper-parámetros de la red
hidden_layers = [2, 10, 10, 10, 1]

# Crear instancia de la NN
u_pinn = NeuralNetwork(hidden_layers)
nparams = sum(p.numel() for p in u_pinn.parameters() if p.requires_grad)
print(f'Number of trainable parameters: {nparams}')

#===============================================================================
# ETAPA 4 Y 5: DEFINICIÓN DE LA FUNCIÓN DE COSTO BASADA EN AUTOGRAD
#===============================================================================
# Error cuadrático medio (Mean Squared Error - MSE)
MSE_func = nn.MSELoss()

def PINN_diffusion_Loss(forward_pass, x_ten, t_ten, 
             lambda1 = 1, lambda2 = 1, lambda3 = 1):

    # ANN output, first and second derivatives
    domain = torch.cat([t_ten, x_ten], dim = 1)
    u = forward_pass(domain)
    u_t = ... # TODO: calculate derivative w/r to t, use t_ten
    u_x = ... # TODO: calculate derivative w/r to x
    u_xx = ... # TODO: calculate second derivative w/r to x 
    
    # TODO: definir PDE loss 
    f_pde = ... # debería usar las variables t_ten, x_ten, u_t y u_xx, además de las funciones torch.sin, torch.exp
    PDE_loss = lambda1 * MSE_func(f_pde, torch.zeros_like(f_pde)) 
    
    # TODO: definir IC loss 
    ic = torch.cat([torch.zeros_like(t_ten), x_ten], dim = 1)
    g_ic = ...
    IC_loss = lambda2 * ...

    # TODO: definir BC x = -1 loss
    bc1 = torch.cat([t_ten, -torch.ones_like(x_ten)], dim = 1)
    h_bc1 = ...
    BC1_loss = lambda3 * ...
    
    # TODO: definir BC x = 1 loss
    bc2 = torch.cat([t_ten, torch.ones_like(x_ten)], dim = 1)
    h_bc2 = ...
    BC2_loss = lambda3 * ...
    
    return PDE_loss + IC_loss + BC1_loss + BC2_loss

#===============================================================================
# ETAPA 6: DEFINICIÓN DEl OPTIMIZADOR
#===============================================================================
learning_rate = 0.001
optimizer = optim.Adam(u_pinn.parameters(), lr=learning_rate,
                        betas= (0.99,0.999), eps = 1e-8)


#===============================================================================
# CICLO DE ENTRENAMIENTO
#===============================================================================
training_iter = 15000

# Inicializar lista para guardar valores de pérdida
loss_values = []

# Empezar timer
start_time = time.time()

# Entrenar red neuronal
for i in range(training_iter):

    optimizer.zero_grad()   # Reinicializar gradientes para iteración de entrenamiento

    # ingresar x, predecir con PINN y obtener pérdida
    loss = PINN_diffusion_Loss(u_pinn, x_ten, t_ten)

    # Agregar actual valor de pérdida a la lista
    loss_values.append(loss.item())

    if i % 1000 == 0:  # Print cada 1000 iteraciones
        print(f"Iteration {i}: Loss {loss.item()}")

    loss.backward() # Paso de retropropagación
    optimizer.step() # Actualizar pesos de la red con optimizador

# Detener timer y obtener tiempo transcurrido
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training time: {elapsed_time} seconds")

In [ ]:
# Graficación
X_ten = torch.tensor(X).float().reshape(-1, 1)
T_ten = torch.tensor(T).float().reshape(-1, 1)
domain_ten = torch.cat([T_ten, X_ten], dim = 1)
U_pred = u_pinn(domain_ten).reshape(dom_samples,dom_samples)

U_true = torch.tensor(U).float()
print(f'Relative error: {relative_l2_error(U_pred, U_true)}')

plot_comparison(U, U_pred, loss_values)

## **Ejercicios**:
1. Agregar caso basado en datos. **Indicación**: Utilice los mismos puntos de colocación de la PINN.
2. Evalúe cómo varía la solución de la PINN aumentando y disminuyendo los pesos `lambdas`.
3. Evalúe cómo varía la solución de la PINN aumentando y disminuyendo la tasa de aprendizaje y el número de iteraciones de entrenamiento.
4. Cambie el número de capas ocultas, neuronas y funciones de activación del modelo de NN.

## **Questions**:
1. ¿Por qué no es necesario incluir datos de la solución en el interior del dominio para resolver el problema de EDP con PINNs?
   <details>
   <summary>Answer</summary>
    En el modelo de difusión, las leyes físicas son bien comprendidas y el problema asociado es bien puesto. El modelo de calor describe cómo una cantidad de interés cambia con el tiempo y cómo esta dinámica se relaciona con el espacio. Las propiedades de los operadores lineales asociados a la EDP de calor hacen que las soluciones al problema de difusión sean únicas y estables, siendo solamente necesarias las condiciones iniciales y de borde para conocerlas completamente al interior del dominio. Este comportamiento hace que no sea necesario tomar un enfoque basado en datos para conocer la solución del problema de difusión, haciendo que la PINN pueda prescindir de información empírica y solo tenga que minimizar residuos asociados a la formulación matemática de la EDP.
   </details>

